<img src="../images/ilmudatapy-logo.png" width="350" align="center">
<br>

<center><h1>Prediksi Pasien Liver dengan Support Vector Machine (SVM)</h1></center>
<hr>

__Halo, Learners!__ Di notebook ini, kita akan mempraktekkan pemodelan <i>machine learning</i> dengan algoritma __Support Vector Machine__ atau __SVM__. Disini kita juga akan mengaplikasikan beberapa jenis visualisasi data dan teknik <i>preprocessing</i> data sebelum dilakukan pemodelan. Kita juga akan coba melakukan perbandingan apabila menggunakan <i>parameter tuning</i> __GridSearchCV__ dan __RandomizedSearchCV__. 

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 25px">
    <ul>
        <li>
            Support Vector Machine (SVM)
        </li>
        <li>
            Dataset
        </li>
        <li>
            Analisis dan visualisasi data
        </li>
        <li>
            Preprocessing
            <ul>
                <li>Menangani missing values</li>
                <li>Menangani duplikat data</li>
                <li>Memisahkan data fitur dan target</li>
                <li>Encoding</li>
                <li>Normalisasi</li>
                <li>Train test split</li>
            </ul>
        </li>
        <li>
            Modeling
            <ul>
                <li>Klasifikasi dengan Support Vector Machine (SVM)</li>
                <li>Evaluasi</li>
            </ul>
        </li>
        <li>
            Parameter Tuning
            <ul>
                <li>GridSearchCV</li>
                <li>RandomizedSearchCV</li>
            </ul>
        </li>
    </ul>
</div>

<hr>
<div class="alert alert-success" style="margin-top: 20px">
    <strong>Catatan:</strong> Untuk menjalankan kode program Python di Jupyter Notebook, klik pada <i>cell</i> yang ingin di-<i>run</i> lalu tekan <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>

<div class="alert alert-danger" style="margin-top: 20px">
    <strong>Warning!:</strong> Jika ada kode program yang <i>error</i> atau output yang dihasilkan tidak sesuai, silahkan <b>Restart & Run All</b> kernel pada bagian menu <b>Kernel</b> di menu bar Jupyter Notebook, atau <b>Restart & Clear Output</b> kernel kemudian jalankan satu per satu <i>cell</i> secara berurutan dari atas ke bawah.
</div>
<hr>

## Support Vector Machine (SVM)

__Support Vector Machine__ atau __SVM__ adalah algoritma <i>machine learning</i> yang cukup populer untuk kasus <i>supervised learning</i>. SVM dapat digunakan untuk kasus klasifikasi maupun regresi, namun kebanyakan digunakan untuk klasifikasi. 

Cara kerja dari algoritma ini adalah dengan membuat <i>best line</i> atau <i>decision boundary</i> yang disebut dengan __hyperplane__ yang membagi ruang n-dimensi ke dalam kelas/kategori sehingga nantinya akan dengan mudah menentukan kelas/kategori data baru. Perhatikan ilustrasi berikut.

![alt text](../images/svm.png 'ilustrasi SVM')

Pada gambar di atas, ada dua kelas/kategori yakni lingkaran yang berwarna merah dan lingkaran yang berwarna kuning. Algoritma SVM akan membuat _hyperplane_ untuk membagi data tersebut. Garis yang berwarna biru merupakan _hyperplane_ terbaik karena memiliki margin yang jauh dari data terdekat, sedangkan garis berwarna abu-abu adalah _hyperplane_ yang buruk karena marginnya terlalu dekat dengan data terdekat pada kategori merah maupun kuning.

<hr>

## Dataset

Dataset yang digunakan adalah dataset <a href='https://archive.ics.uci.edu/ml/datasets/ILPD+(Indian+Liver+Patient+Dataset)'>ILPD (Indian Liver Patient Dataset)</a> yang berasal dari UCI Machine Learning Repository dengan informasi detail tentang tiap kolom (terurut dari awal sampai akhir) sebagai berikut:

__Attribute Information:__

1. __Age:__ Age of the patient
2. __Gender:__ Gender of the patient
3. __TB:__ Total Bilirubin
4. __DB:__ Direct Bilirubin
5. __Alkphos:__ Alkaline Phosphotase
6. __Sgpt:__ Alamine Aminotransferase
7. __Sgot:__ Aspartate Aminotransferase
8. __TP:__ Total Protiens
9. __ALB:__ Albumin
10. __A/G Ratio:__ Albumin and Globulin Ratio
11. Selector field used to split the data into two sets (labeled by the experts)

Kita akan membuat model <i>machine learning</i> untuk memprediksi kelas data baru apakah termasuk pasien liver atau bukan menggunakan algoritma Support Vector Machine (SVM).

Seperti biasa pertama kita <i>import</i> <i>library</i> yang akan digunakan.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Mengabaikan warning yang mungkin terjadi
import warnings
warnings.filterwarnings('ignore')

Selanjutnya kita <i>load</i> dataset CSV ke dalam dataframe Pandas dengan <code>read_csv()</code>. Dataset ini belum memiliki <i>header</i> atau nama kolom, karena itu kita akan mendefinisikan terlebih dahulu nama kolomnya agar lebih mudah untuk proses analisis data.

In [ ]:
# Mendefinisikan nama kolom
column_names = ['Age','Gender','Total Bilirubin','Direct Bilirubin','Alkaline Phosphotase','SGPT','SGOT','Total Protein',
                'Albumin','Albumin and Globulin Ratio','Class']

# Load dataset
df = pd.read_csv('../datasets/Indian-Liver-Patient-Dataset-(ILPD).csv', names = column_names)
df.head(10)

<hr>

## Analisis dan visualisasi data

Biasanya yang pertama kali dicek adalah informasi singkat mengenai dataset tersebut. Nah, kita dapat menggunakan <code>info()</code> untuk mendapatkannya.

In [ ]:
# Menampilkan info dataset

df.info()

Dari hasil <code>info()</code>, dapat kita liha bahwa pada kolom <code>Albumin and Globulin Ratio</code> jumlah datanya lebih sedikit dari kolom lainnya. Ini berarti ada <i>missing values</i> disitu.

Selanjutnya mari kita cek statistik deskriptifnya dengan <code>describe()</code>.

In [ ]:
# Menampilkan statistik deskriptif data

df.describe()

<code>describe()</code> menampilkan statistik dari dataset seperti jumlah data (__count__), rata-rata (__mean__), standar deviasi (__std__), nilai minimum (__min__), kuartil 1 (__25%__), median/kuartil 2 (__50%__), kuartil 3 (__75%__), dan nilai maksimum (__max__) untuk tiap kolom numerik.

Lalu mari kita cek jumlah data untuk tiap kategori pada kolom target yaitu <code>Class</code> dengan <code>value_counts()</code>.

In [ ]:
# Menampilkan jumlah data untuk tiap kategori pada kolom 'Class'

df['Class'].value_counts()

Dari hasil di atas, ada __416__ data yang termasuk dalam <i>class</i> <b>1</b> (<b>Liver Patient</b>) dan <b>167</b> di <i>class</i> <b>2</b> (<b>Non Liver Patient</b>).

Sekarang mari kita visualisasikan kolom <code>Class</code> berdasarkan jenis kelaminnya.

In [ ]:
# Visualisasi data dengan countplot 

sns.countplot(x='Class', hue='Gender', data=df, palette='Set1')

Ternyata untuk masing-masing <i>class</i> jumlah pasien laki-laki lebih banyak dari pasien perempuan.

Kita juga dapat membuat <i>scatter plot</i>-nya. Misalnya disini kita ingin melihat sebaran 100 data pertama berdasarkan kolom <code>Age</code> dan <code>Albumin</code>.

In [ ]:
# Visualisasi 100 data pertama dengan scatter plot untuk kolom 'Class'

x = df[df['Class'] == 1][0:100].plot(kind='scatter', x='Age', y='Albumin', color='SteelBlue', label='Liver Patient');

df[df['Class'] == 2][0:100].plot(kind='scatter', x='Age', y='Albumin', color='Gold', label='Non Liver Patient', ax=x);
plt.show()

Kita juga dapat menggunakan histogram untuk melihat sebaran data tiap kolom.

In [ ]:
# Menampilkan histogram dari kolom fitur numerik

fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(16, 10)) 

# Menambahkan subplot dengan indexing
ax0 = fig.add_subplot(ax[0,0]) 
ax1 = fig.add_subplot(ax[0,1])  
ax2 = fig.add_subplot(ax[0,2])  
ax3 = fig.add_subplot(ax[1,0]) 
ax4 = fig.add_subplot(ax[1,1])  
ax5 = fig.add_subplot(ax[1,2]) 
ax6 = fig.add_subplot(ax[2,0]) 
ax7 = fig.add_subplot(ax[2,1])  
ax8 = fig.add_subplot(ax[2,2]) 

df.hist(column='Age', bins=50, ax=ax0)
df.hist(column='Total Bilirubin', bins=50, ax=ax1)
df.hist(column='Direct Bilirubin', bins=50, ax=ax2)
df.hist(column='Alkaline Phosphotase', bins=50, ax=ax3)
df.hist(column='SGPT', bins=50, ax=ax4)
df.hist(column='SGOT', bins=50, ax=ax5)
df.hist(column='Total Protein', bins=50, ax=ax6)
df.hist(column='Albumin', bins=50, ax=ax7)
df.hist(column='Albumin and Globulin Ratio', bins=50, ax=ax8)

plt.subplots_adjust(wspace=0.2, hspace=0.4)
plt.show()

<hr>

## Preprocessing

Mari kita cek dimensi data dengan atribut <code>shape</code>.

In [ ]:
# Mengecek dimensi data

df.shape

### Menangani missing values

Seperti yang telah kita ketahui sebelumnya bahwa ada <i>missing values</i> dalam dataset tersebut. Sekarang mari kita lihat ada berapa <i>missing values</i> pada dataset ini dengan <code>isnull().sum()</code>.

In [ ]:
# Mengecek missing values pada tiap kolom

df.isnull().sum()

Ternyata hanya ada __4__ <i>missing values</i> pada kolom <code>Albumin dan Globulin Ratio</code>.

Selanjutnya mari kita isi nilai yang hilang tersebut dengan <code>mean()</code> atau nilai rata-rata dari kolom tersebut.

In [ ]:
# Mengisi missing values dengan nilai rata-rata 

df['Albumin and Globulin Ratio'].fillna(df['Albumin and Globulin Ratio'].mean(), inplace=True)

Cek lagi jumlah <i>missing values</i>.

In [ ]:
# Mengecek kembali missing values

df.isnull().sum()

Sekarang sudah tidak ada <i>missing values</i>.

### Menangani duplikat data

Kita akan mengecek apakah ada duplikat data pada dataframe <code>df</code> dengan <code>duplicated().values.any()</code>.

In [ ]:
# Mengecek duplikat data

df.duplicated().values.any()

Ternyata ada duplikat data dalam dataset ini. Sekarang mari kita hapus duplikat data tersebut dengan <code>drop_duplicates()</code>.

In [ ]:
# Menghapus duplikat data

df = df.drop_duplicates()

Lalu cek lagi dimensi data setelah menghapus duplikat.

In [ ]:
# Mengecek dimensi data setelah menghapus duplikat

df.shape

Dimensi data sekarang adalah (__570, 11__) yang berarti berkurang __13__ baris data dari yang sebelumnya 583.

### Memisahkan data fitur dan target

Selanjutnya kita akan memisahkan data fitur dan target. Data targetnya adalah kolom <code>Class</code> dan sisa kolom lainnya merupakan data fitur, sehingga untuk mendefinisikan data fitur, kita hanya perlu menghapus kolom <code>Class</code>.

In [ ]:
# Mendefinisikan data fitur dan target

df_features = df.drop('Class', axis=1)
df_target = df['Class']

Mari kita tampilkan <code>df_features</code>.

In [ ]:
# Menampilkan data fitur

df_features.head()

### Encoding

Dari 10 kolom yang akan digunakan sebagai fitur, kolom <code>Gender</code> masih bertipe non-numerik. Oleh karena itu, disini kita akan melakukan <i>encoding</i> pada kolom tersebut dengan <code>.cat.codes</code> dengan mengubah tipe datanya terlebih dahulu menjadi <i>category</i>.

In [ ]:
# Mengubah tipe data 'Gender' menjadi category
df_features['Gender'] = df_features['Gender'].astype('category')

# Encoding data
df_features['Gender'] = df_features['Gender'].cat.codes
df_features.head()

### Normalisasi

Seperti yang kita lihat bahwa <i>range</i> angka tiap kolom berbeda jauh, sehingga kita perlu melakukan normalisasi terlebih dahulu terhadap dataframe <code>df_features</code>.

Disini kita akan menggunakan <code>StandardScaler</code> untuk normalisasi data.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Mendefinisikan nama kolom fitur
cols = list(df_features.columns)

# Normalisasi data dengan StandardScaler
df_features_scaled = pd.DataFrame(data = df_features)
df_features_scaled[cols] = StandardScaler().fit_transform(df_features[cols])
df_features_scaled.head()

Setelah dataframe fitur siap digunakan, sekarang kita gabungkan lagi dengan data targetnya.

In [ ]:
# Menggabungkan kembali dataframe fitur dan target

df_join = pd.concat([df_features_scaled, df_target], axis=1)
df_join.head()

### Train test split

Selanjutnya kita akan memisahkan <code>df_join</code> menjadi data latih dan data uji dengan <code>train_test_split</code> dengan persentase 80% data latih dan 20% data uji.

In [ ]:
from sklearn.model_selection import train_test_split

# Mendefinisikan fitur dan target
X = df_join.iloc[:,:-1] 
y = df_join['Class']

# Membagi data latih dan data uji dengan train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# Menampilkan dimensi data latih dan data uji
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

<hr>

## Modeling 

### Klasifikasi dengan Support Vector Machine (SVM)

Untuk masalah klasifikasi, kita dapat menggunakan __Support Vector Classification__ atau __SVC__ dari SVM. 

Tahapannya sama dengan proses klasifikasi dengan algoritma lainnya, yaitu mendefinisikan model <code>SVC()</code>, melatihnya dengan <code>fit()</code>, kemudian menguji model dengan melakukan prediksi terhadap data test dengan <code>predict()</code>.

In [ ]:
from sklearn.svm import SVC

# Melatih model
model = SVC(gamma='scale').fit(X_train, y_train)

# Menguji model / memprediksi dengan X_test
y_pred = model.predict(X_test)

### Evaluasi

Selanjutnya mari kita evaluasi hasil model tersebut dengan <code>accuracy_score()</code> untuk <i>train set</i> dan <i>test set</i>-nya.

In [ ]:
from sklearn.metrics import accuracy_score

# Menampilkan akurasi data latih dan data uji
print('Akurasi Train set: %.3f' % accuracy_score(y_train, model.predict(X_train)))
print('Akurasi Test set: %.3f' % accuracy_score(y_test, y_pred))

<hr>

## Parameter Tuning

Parameter tuning adalah sebuah proses mencari kombinasi parameter yang akan menghasilkan akurasi terbaik. Di Sklearn, ada dua <i>package</i> dari <i>library</i> __model_selection__ yang dapat digunakan untuk proses parameter tuning, yaitu __GridSearchCV__ dan __RandomizedSearchCV__.

### GridSearchCV

<code>GridSearchCV</code> adalah salah satu teknik parameter tuning yang cara kerjanya yaitu dengan mencoba semua kombinasi parameter yang didefinisikan.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Mendefinisikan parameter yang ingin dicoba
param_grid = {'gamma': ['auto', 'scale'], 
             'kernel': ['linear', 'sigmoid', 'poly', 'rbf'], 
             'degree': np.arange(1,6)}

# Modeling dengan SVM + Grid Search
model = SVC()
gscv = GridSearchCV(model, param_grid, scoring='accuracy', cv=10)
gscv.fit(X_train, y_train)

Untuk menampilkan hasil parameter terbaik dapat menggunakan atribut <code>.best_param_</code>.

In [ ]:
# Menampilkan hasil parameter terbaik

print('Parameter terbaik hasil Grid Search: ', gscv.best_params_)

Untuk menampilkan hasil akurasi terbaik dapat menggunakan atribut <code>.best_score_</code>.

In [ ]:
# Menampilkan akurasi data latih

print('Akurasi Train set: %.3f'% gscv.best_score_)

Mari kita uji model dengan data uji <code>X_test</code> kemudian tampilkan skor akurasinya.

In [ ]:
# Menguji model dengan X_test
y_pred_gscv = gscv.predict(X_test)

# Menampilkan akurasi hasil pengujian
print('Akurasi Test set: %.3f'% accuracy_score(y_test, y_pred_gscv))

### RandomizedSearchCV

<code>RandomizedSearchCV</code> adalah salah satu teknik parameter tuning dimana parameter yang uji diambil secara acak dari parameter yang sudah didefinisikan. 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Mendefinisikan parameter yang ingin dicoba
param_grid = {'gamma': ['auto', 'scale'], 
             'kernel': ['linear', 'sigmoid', 'poly', 'rbf'], 
             'degree': np.arange(1,6)}

# Modeling dengan SVM + Randomized Search
model = SVC()
rscv = RandomizedSearchCV(model, param_grid, scoring='accuracy', cv=10)
rscv.fit(X_train, y_train)

Mari kita tampilkan parameter terbaik.

In [ ]:
# Menampilkan parameter terbaik

print('Parameter terbaik hasil Randomized Search: ', rscv.best_params_)

Selanjutnya, kita tampilkan skor terbaik.

In [ ]:
# Menampilkan hasil akurasi data latih

print('Akurasi Train set: %.3f'% rscv.best_score_)

Kemudian kita uji dengan data test <code>X_test</code> dan <i>print</i> hasil akurasinya.

In [ ]:
# Menguji model dengan X_test
y_pred_rscv = rscv.predict(X_test)

# Menampilkan akurasi hasil pengujian
print('Akurasi Test set: %.3f'% accuracy_score(y_test, y_pred_rscv))

Hasil RandomizedSearchCV dapat berubah-ubah karena di-<i>generate</i> secara random.

<hr>

Copyright @ <a href="https://ilmudatapy.com/">ilmudatapy.com</a>